<a href="https://colab.research.google.com/github/JPxYang/PythonLearn/blob/main/%E3%80%8C0_Langchain_Agent_%E9%87%8D%E6%96%B0%E8%AA%8D%E8%AD%98_Agent_ipynb%E3%80%8D_%E5%AD%B8%E7%BF%92202308.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 重新認識認識 Agent

Agent 是一個推理引擎，旨在做出決策、選擇工具並採取行動以實現特定的、通常是預定義的目標或一組目標。Agent 自主運行，很少需要人工干預其操作。
正是正確的強大工具以及自力更生，目前使 Agent 吸引了如此多的注意力。它們當然是我們的未來，值得熟悉這項技術。

本章節中，將帶領各位學員，如何熟練掌握 Agent 使用技巧與知識

* Agent 基礎認識
* Agent 多種 Type 介紹，如：ReAct、zero shot
* 客製化 Agent
* 客製化 Tool
* 分享多個有趣案例

官網連結: [Agent](https://python.langchain.com/en/latest/modules/agents.html)

![Qoo](https://imgur.com/VxVFEUl.png)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 建立環境與設置參數

### 安裝套件包

In [3]:
!pip install langchain
!pip install openai
!pip install requests
!pip install duckduckgo-search
!pip install wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.1.1
    Uninstalling pydantic-2.1.1:
      Successfully uninstalled pydantic-2.1.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 94.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


### 設定專案 Log 資訊


In [4]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### 環境參數建置

In [5]:
import os
from getpass import getpass

### OPENAI 環節

In [6]:
if os.getenv("OPENAI_API_KEY") is None:
  os.environ["OPENAI_API_KEY"] = getpass("可以從以下網站找到 Api-key: https://platform.openai.com/account/api-keys\n")
assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "貌似不是合理的 OpenAI API key"
print("OpenAI API key configured")

可以從以下網站找到 Api-key: https://platform.openai.com/account/api-keys
··········
OpenAI API key configured


### WA_環節


In [7]:
if os.getenv("WA_API_KEY") is None:
  os.environ["WA_API_KEY"] = getpass("可以從以下網站找到 Api-key: https://www.wolframalpha.com/\n")
print("WA API key configured")

可以從以下網站找到 Api-key: https://www.wolframalpha.com/
··········
WA API key configured


## 初步操作 Agent



### 暖暖手 - 建構簡單 LLM Chain

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# Initiate our LLM - default is 'gpt-3.5-turbo'
llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0, request_timeout=120)

#
prompt = PromptTemplate(
  input_variables=["query"],
  template="You are New Native Internal Bot. Help users with their important tasks, like a professor in a particular field. Query: {query}"
)

#
llm_chain = LLMChain(llm=llm, prompt=prompt)

### 測試 Chain 效果

In [9]:
llm_chain.run("what is (4.5*2.1)^2.2?")

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..


'To calculate (4.5 * 2.1) raised to the power of 2.2, you can follow these steps:\n\nStep 1: Calculate the product of 4.5 and 2.1:\n4.5 * 2.1 = 9.45\n\nStep 2: Raise the result from step 1 to the power of 2.2:\n9.45^2.2 ≈ 38.718\n\nTherefore, (4.5 * 2.1)^2.2 is approximately equal to 38.718.'

### 替 Agent 增加聯外工具

In [10]:
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import Tool

search = DuckDuckGoSearchRun()

# Web Search Tool
search_tool = Tool(
    name = "Web Search",
    func=search.run,
    description="A useful tool for searching the Internet to find information on world events, issues, etc. Worth using for general topics. Use precise questions."
)

In [11]:
import requests

class WA:
  """
    Wolfram|Alpha API
  """
  def __init__(self, app_id):
    self.url = f"http://api.wolframalpha.com/v1/result?appid={app_id}&i="

  def run(self, query):
    query = query.replace("+", " plus ").replace("-", " minus ") # '+' and '-' are used in URI and cannot be used in request
    result = requests.post(f"{self.url}{query}")

    if not result.ok:
      raise Exception("Cannot call WA API.")

    return result.text

In [12]:
wa = WA(app_id=os.environ["WA_API_KEY"])

wa_tool = Tool(
    name="Wolfram|Alpha API",
    func=wa.run,
    description="Wolfram|Alpha API. It's super powerful Math tool. Use it for simple & complex math tasks."
)

### 建立 Agent 並測試聯外工具

In [13]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=[wa_tool, search_tool],
    llm=llm,
    verbose=True, # I will use verbose=True to check process of choosing tool by Agent
    max_iterations=3
)

In [14]:
response = agent("what is (4.5*2.1)^2.2?")
print(f"Final answer: {response['output']}")



> Entering new AgentExecutor chain...
This is a math calculation that can be easily solved using the Wolfram|Alpha API.
Action: Wolfram|Alpha API
Action Input: (4.5*2.1)^2.2
Observation: 139.943
Thought:I now know the final answer
Final Answer: 139.943

> Finished chain.
Final answer: 139.943


## 慢著，Agent 與 Chain 有何不同？

如果我們查看 Agent 的邏輯和剛剛 printout 的提示，我們將看到一些明顯的差異。首先，我們有提示中包含的工具。其次，我們有一個思維過程，以前是即時的 Chain，但現在涉及“Though”，“Action”，“Action input”，“Obervation”。這到底是怎麼回事？

現在只要說LLM現在有能力“推理”如何最好地使用工具來解決我們的查詢，並且可以以智慧的方式將它們組合在一起，只需對每個工具進行簡要描述。如果您想詳細瞭解此範例 （MRKL），請[參閱本文](https://arxiv.org/pdf/2205.00445.pdf)。

這就是我們將附加代理已經執行的每個想法或操作的地方。這樣，在每個時間點，智慧體都會知道它發現了什麼，並能夠繼續其思維過程。換句話說，在使用工具后，它會將其想法和觀察添加到暫存器中，並從那裡拾取。

![link text](https://imgur.com/eDiWmja.jpg)

## Agent Types 認識

在本節中，我們將回顧幾個 Agent，看看他們如何“思考”以及他們可以做什麼。

使用langchain的 Agent 中涉及三個變數：

* 定義工具(tool)
* 定義大型語言模型(LLM)
* 定義Agent 種類 (Agent Type)

這在langchain中非常容易做到，正如我們將在下面的範例中看到的那樣。

LangChain [多種 Type 介紹](https://python.langchain.com/en/latest/modules/agents/agents/agent_types.html)

Langchain [github Type](https://github.com/hwchase17/langchain/blob/d9fcc45d057e3c8148dc82efd1150c383fcb245e/langchain/agents/agent_types.py#L4)

###認識 Agent 種類 - Agnet Type #1: Zero Shot React

我們首先需要初始化我們將在本例中使用的工具。在這裡，我們希望我們的代理做數學，所以我們將使用“llm-math”，這是我們解決數學問題的工具。這是我們可以使用 load_tools 實用程式載入的幾個工具之一 - 您可以通過查看文件來查看更多資訊。

LangChain 官方連結：[Agent_type](https://python.langchain.com/en/latest/modules/agents/agents/agent_types.html)

In [ ]:
from langchain.agents import load_tools

tools = load_tools(
    ["llm-math"],
    llm=llm
)

zero_shot_react_agent = initialize_agent(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True, # I will use verbose=True to check process of choosing tool by Agent
    max_iterations=3
)

response = zero_shot_react_agent("what is (4.5*2.1)^2.2?")
print(f"Final answer: {response['output']}")



> Entering new AgentExecutor chain...
I need to use the calculator to solve this problem.
Action: Calculator
Action Input: (4.5*2.1)^2.2
Observation: Answer: 139.94261298333066
Thought:I now know the final answer.
Final Answer: 139.94261298333066

> Finished chain.
Final answer: 139.94261298333066


#### 顯示查看 Agent 使用的 Prompt

In [ ]:
print(zero_shot_react_agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


###認識 Agent 種類 - Agent Type #2: Chat Conversational ReAct

前面章節當中，我們已經認識到 Zerp Shot Agent，是挺有趣的，但是該 Agent 本質為 Stateless，缺乏記憶的。我們常用的場景中是希望這名聊天對象記得我們的談話，為此該怎麼做呢？我們需要使用 conversational react agemt

In [ ]:
from langchain.agents import load_tools

tools = load_tools(
    ["llm-math"],
    llm=llm
)

#### 添加記憶

添加 Memory 的方式就很多了，Langchain 就提供以下幾種

* ConversationBufferMemory 暫存對話
* ConversationBufferWindowMemory 保留一段對話的暫存對話
* MongoDBChatMessageHistory 使用 MangoDB 紀錄對話資訊

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

conversation_memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=5,
    return_messages=True
)

conversational_agent = initialize_agent(
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=conversation_memory,
)

In [ ]:
response = conversational_agent("what is (4.5*2.1)^2.2?")
print(f"Final answer: {response['output']}")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "139.94261298333066"
}

> Finished chain.
Final answer: 139.94261298333066


#### 顯示查看 Agent 使用的 Prompt

In [ ]:
print(conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful system that can help with a wide range of task

[點我查看更多內容資訊](https://arxiv.org/pdf/2210.03629.pdf)

### 認識 Agent 種類 - Agent Type #3: ReAct DocStore

這種類型的代理類似於我們目前看到的代理，但它包括與文檔存儲的交互。它將有兩個且只有兩個工具可供使用：「Search」和「lookup」。

使用 search ，它將調出相關文章，使用 lookup， Agent 將在文章中找到正確的資訊。這可能在示例中最容易看到：

In [ ]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

docstore=DocstoreExplorer(Wikipedia())

tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia'
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]

conversation_memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=5,
    return_messages=True
)

docstore_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.REACT_DOCSTORE,
    verbose=True,
    max_iterations=3,
    memory=conversation_memory
)

In [ ]:
response = docstore_agent("What were Archimedes' last words?")
print(f"Final answer: {response['output']}")



> Entering new AgentExecutor chain...
Thought: I need to search for Archimedes' last words and find out what they were.
Action: Search[Archimedes' last words]
Observation: A person's last words, their final articulated words stated prior to death or as death approaches, are often recorded because of the decedent's fame, but sometimes because of interest in the statement itself. (People dying of illness are frequently inarticulate at the end, and in such cases their actual last utterances may not be recorded or considered very important.) Last words may be recorded accurately, or, for a variety of reasons, may not. Reasons can include simple error or deliberate intent. Even if reported wrongly, putative last words can constitute an important part of the perceived historical records or demonstration of cultural attitudes toward death at the time.Charles Darwin, for example, was reported to have disavowed his theory of evolution in favor of traditional religious faith at his death. This

## 總結

目前為止學習到在 LangChain 當中多種 Agent Type 並且了改每一種思維模式，接下來我們將讓 Agent 更加聰明些，目標如下

* 新增 tool
* 新增 agent